### **1. Import Package**

In [ ]:
!pip install pandas-plink
!pip install dask-ml --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.4/79.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 56.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 57.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.9/292.9 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: zstandard
    Found existing installation: zstandard 0.25.0
    Uninstalling zstandard-0.25.0:
      Successfully uninstalled zstandard-0.25.0
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incom

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.0/150.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.4/259.4 kB 19.9 MB/s eta 0:00:00
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
^C


In [ ]:
from pandas_plink import read_plink
from sklearn.metrics import pairwise_distances
import dask.array as da
import numpy as np
import numpy as np
import pandas as pd
import random
import time

###**2. Load Data PLINK**

In [ ]:
!gunzip -k /content/core_v0.7.bed.gz
!gunzip -k /content/core_v0.7.bim.gz
!gunzip -k /content/core_v0.7.fam.gz

In [ ]:
bim, fam, bed = read_plink("core_v0.7")

# bim → informasi SNP (chrom, posisi, allele)
# fam → informasi sampel (ID individu)
# bed → matrix genotipe ukuran: variants × samples
# Sandi Genotipe:
# 0 = homozigot referensi
# 1 = heterozigot
# 2 = homozigot alternatif
# -1 = missing

print("SNPs:", bed.shape[0])
print("Samples:", bed.shape[1])
# print(type(bed))
# print(bed)

small_view = bed[:5, :].compute()
print(small_view)
print(fam.head())
print(bim.head())

Mapping files: 100%|██████████| 3/3 [00:02<00:00,  1.27it/s]


SNPs: 404388
Samples: 3024
[[2. 2. 2. ... 0. 0. 2.]
 [2. 2. 2. ... 0. 0. 2.]
 [2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 2. 2. 2.]
 [2. 2. 2. ... 0. 0. 2.]]
    fid   iid father mother gender trait  i
0  B001  B001      0      0      0    -9  0
1  B002  B002      0      0      0    -9  1
2  B003  B003      0      0      0    -9  2
3  B004  B004      0      0      0    -9  3
4  B005  B005      0      0      0    -9  4
  chrom   snp   cm   pos a0 a1  i
0     1  1178  0.0  1178  T  G  0
1     1  1203  0.0  1203  C  T  1
2     1  1248  0.0  1248  A  G  2
3     1  1282  0.0  1282  A  G  3
4     1  1299  0.0  1299  C  T  4


### **3. Preprocessing**

In [ ]:
# Matriks Genotipe
G0 = bed       # (variants × samples)

# Mengubah Missing Genotype Menjadi NaN
G_filled = da.where(G0 < 0, da.nan, G0)
# Mengimputasi Genotipe yang Missing dengan Rata-rata SNP
snp_mean = da.nanmean(G_filled, axis=1, keepdims=True)
G_filled = da.where(da.isnan(G_filled), snp_mean, G_filled)

# Standardisasi per SNP
snp_std = da.std(G_filled, axis=1, keepdims=True)
G_std = (G_filled - snp_mean) / (snp_std + 1e-8)


### **4. Pembuatan Genomic Relationship Matrix (GRM)**

In [ ]:
# Hitung M = jumlah SNP
M = G_std.shape[0]

# Membuat GRM (Genomic Relationship Matrix)
# GRM = “kinship matrix genomik” → Mengukur kedekatan genetik antar-individu.
GRM = (G_std.T @ G_std) / M
GRM = GRM.compute()

### **5. PCA (Principal Component Analysis)**



---
Fungsi PCA untuk Parent Selection dalam breeding atau optimisasi pemilihan induk (parent selection):
1.   Mengidentifikasi struktur populasi
2.   Meminimalkan inbreeding
3.   Meningkatkan diversitas genetik

In [ ]:
# melalui eigen decomposition
vals, vecs = np.linalg.eigh(GRM)

# Mengurutkan eigenvalue dari besar → kecil
idx = np.argsort(vals)[::-1]
# Menyusun ulang eigenvalue dan PC sesuai urutan
eigenvalues = vals[idx]
PC = vecs[:, idx]

# mengambil 20 Principal Components teratas
PC20 = PC[:, :20]
PC20.shape

(3024, 100)

### **6. Hitung jarak genetik dari PC20**

In [ ]:
# Menghitung jarak genetik antar calon induk
dist_matrix = pairwise_distances(PC20)

print(dist_matrix.shape)   # (n_samples, n_samples)
print(fam.shape)           # (n_samples, 7)


(3024, 3024)
(3024, 7)


### **7. Genetic Algorithm (GA)**

In [ ]:
# ---------------------------------------------------
# Inputan
# dist_matrix = pairwise_distances(PC20) >jarak genetik antar individu hasil PCA
# fam = fam dari read_plink(...)
# ---------------------------------------------------

n_samples = dist_matrix.shape[0]
sample_ids = fam.iid.values

# -------------------------------
# 1. Fitness function
# Fitness → jarak genetik antar pasangan.
# -------------------------------
def fitness(pair):
    i, j = pair     #sepasang individu.
    return float(dist_matrix[i, j])     # Semakin besar jaraknya → semakin “bagus” (mengurangi inbreeding).

# -------------------------------
# 2. Komponen Genetic Algorithm (GA)
# -------------------------------
# Membuat pasangan acak
def random_pair():
    a = random.randrange(n_samples)
    b = random.randrange(n_samples)
    while b == a:
        b = random.randrange(n_samples)
    return (a, b)

def crossover(p1, p2):
    # permisalan sederhana Menggabungkan dua pasangan “parent” → menghasilkan pasangan baru.
    return (p1[0], p2[1])

# Mengubah anggota pasangan secara acak dengan probabilitas mut_rate
def mutate(pair, mut_rate=0.1):
    a, b = pair
    if random.random() < mut_rate:
        a = random.randrange(n_samples)
    if random.random() < mut_rate:
        b = random.randrange(n_samples)
    while b == a:
        b = random.randrange(n_samples)
    return (a, b)

# Memilih kandidat terbaik dari k acak dalam populasi → meniru seleksi alam.
def tournament_select(pop, fitness_scores, k=3):
    idxs = random.sample(range(len(pop)), k)
    best = max(idxs, key=lambda i: fitness_scores[i])
    return pop[best]

# -------------------------------
# 3. GA Hyperparameters
# -------------------------------
POP = 200       # populasi pasangan
GEN = 200       # jumlah generasi
ELITE = int(0.1 * POP) # elitism: top 10% langsung lanjut ke generasi berikut
K_TOUR = 3      # ukuran turnamen
MUT = 0.1       # probabilitas mutasi

# -------------------------------
# 4. Initial population
# -------------------------------
population = [random_pair() for _ in range(POP)]

print("Start GA...")
t0 = time.time()

# Evolusi GA
for g in range(GEN):

    scores = [fitness(ind) for ind in population]

    # sort by score desc
    order = np.argsort(scores)[::-1]
    population = [population[i] for i in order]
    scores = [scores[i] for i in order]

    if g % 10 == 0:
        print(f"Gen {g:03d} | best={scores[0]:.4f} | mean={np.mean(scores):.4f}")

    # elitism
    new_population = population[:ELITE]

    # fill with mating
    while len(new_population) < POP:
        p1 = tournament_select(population, scores, k=K_TOUR)
        p2 = tournament_select(population, scores, k=K_TOUR)
        child = crossover(p1, p2)
        child = mutate(child, MUT)
        new_population.append(child)

    population = new_population

t1 = time.time()
print(f"GA done in {t1-t0:.2f}s")

# -------------------------------
# 5. Ambil 10 pasangan terbaik
# -------------------------------
final = [(ind, fitness(ind)) for ind in population]
final = sorted(final, key=lambda x: x[1], reverse=True)

top = []
seen = set()

for pair, sc in final:
    if pair not in seen and (pair[1], pair[0]) not in seen:
        top.append((pair, sc))
        seen.add(pair)
    if len(top) == 10:
        break

# -------------------------------
# 6. Buat DataFrame hasil
# -------------------------------
rows = []
for (i, j), sc in top:
    rows.append({
        "idx1": i,
        "idx2": j,
        "ID1": sample_ids[i],
        "ID2": sample_ids[j],
        "distance": sc
    })

df_results = pd.DataFrame(rows)
df_results


Start GA...
Gen 000 | best=0.4075 | mean=0.2473
Gen 010 | best=0.5807 | mean=0.4839
Gen 020 | best=0.5807 | mean=0.5511
Gen 030 | best=0.5878 | mean=0.5596
Gen 040 | best=0.6026 | mean=0.5690
Gen 050 | best=0.6026 | mean=0.5767
Gen 060 | best=0.6026 | mean=0.5829
Gen 070 | best=0.6133 | mean=0.5870
Gen 080 | best=0.6133 | mean=0.5935
Gen 090 | best=0.6133 | mean=0.5903
Gen 100 | best=0.7421 | mean=0.6892
Gen 110 | best=0.7421 | mean=0.7013
Gen 120 | best=0.7421 | mean=0.6939
Gen 130 | best=0.7421 | mean=0.7015
Gen 140 | best=0.7421 | mean=0.6965
Gen 150 | best=0.7421 | mean=0.6868
Gen 160 | best=0.7421 | mean=0.6922
Gen 170 | best=0.7421 | mean=0.7009
Gen 180 | best=0.7421 | mean=0.7176
Gen 190 | best=0.7421 | mean=0.7123
GA done in 0.55s


,idx1,idx2,ID1,ID2,distance
0,529,2397,CX402,IRIS_313-11542,0.742107
1,529,1923,CX402,IRIS_313-11032,0.648987
2,529,2864,CX402,IRIS_313-12080,0.647719
3,529,3002,CX402,IRIS_313-12349,0.628693
4,529,2317,CX402,IRIS_313-11439,0.627631
5,529,2998,CX402,IRIS_313-12345,0.627590
6,529,1998,CX402,IRIS_313-11107,0.624989
7,529,2322,CX402,IRIS_313-11446,0.623382
8,529,1243,CX402,IRIS_313-9851,0.620175
9,529,1109,CX402,IRIS_313-9402,0.619908
